<a href="https://colab.research.google.com/github/google-research/skai/blob/skai-colab-0000004/src/colab/Run_SKAI_Colab_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SKAI is the limit 🚀
*Assessing Post-Disaster Damage 🏚️ from Satellite Imagery 🛰️ using Semi-Supervised Learning Techniques 📔*

*Contributors:  Amine Baha (1), Joseph Xu (2), Jihyeon Lee (2), Tomer Shekel (2), Fiona Huang (1)*

*Co-developed by (1) WFP Innovation Accelerator and (2) Google Research AI, January 2023*

## Intro 🏹

WFP partnered with Google Research to set up **SKAI**, a humanitarian response mapping solution powered by artificial intelligence — an approach that combines statistical methods, data and modern computing techniques to automate specific tasks. SKAI assesses damage to buildings by applying computer vision — computer algorithms that can interpret information extracted from visual materials such as, in this case, **satellite images of areas impacted by conflict, climate events, or other disasters**.

![Skai Logo](https://storage.googleapis.com/skai-public/skai_logo.png)

The type of machine learning used in SKAI, learns from a small number of labeled and a large number of unlabeled images of affected buildings. SKAI uses a ***semi-supervised learning technique*** that reduces the required number of labeled examples by an order of magnitude. As such, SKAI models typically *only need a couple hundred labeled examples* to achieve high accuracy, significantly improving the speed at which accurate results can be obtained.

Google Research presented this novel application of semi-supervised learning (SSL) to train models for damage assessment with a minimal amount of labeled data and large amount of unlabeled data in [June 2020](https://ai.googleblog.com/2020/06/machine-learning-based-damage.html). Using the state-of-the-art methods including [MixMatch](https://arxiv.org/abs/1905.02249) and [FixMatch](https://arxiv.org/abs/2001.07685), they compare the performance with supervised baseline for the 2010 Haiti earthquake, 2017 Santa Rosa wildfire, and 2016 armed conflict in Syria.

![SSL Approach](https://storage.googleapis.com/skai-public/ssl_diagram.png)

The [paper](https://arxiv.org/abs/2011.14004) published by *Jihyeon Lee, Joseph Z. Xu, Kihyuk Sohn, Wenhan Lu, David Berthelot, Izzeddin Gur, Pranav Khaitan, Ke-Wei, Huang, Kyriacos Koupparis, Bernhard Kowatsch* shows how models trained with SSL methods can reach fully supervised performance despite using only a fraction of labeled data.


## Notebook Setup 📓

**Please refer to the [SKAI Colab Notebooks Instructions](/docs/colab_instructions.md) before running this Colab notebook.**
**Before running this Colab notebook, we recommend to initialize your kernel using [Initialize SKAI Colab Kernel Notebook](https://github.com/google-research/skai/blob/main/src/colab/Initialize_SKAI_Colab_Kernel.ipynb).**

In [ ]:
import os
import datetime

#@title Please run this cell first!

#@markdown Specify the parameters to set up your Colab notebook. They should be the same that the ones used during the initialization of the Colab kernel
#############################################
### CODE SETTING - ENVIRONMENT ACTIVATION ###
#############################################
#@markdown ---
#@markdown Please enter the path to the **git repository** and **colab workspace directory** to use:

#@markdown ---
SKAI_CODE_DIR = "/content/skai_src"  #@param {type:"string"}
SKAI_VENV_DIR = "/content/skai_env"  #@param {type:"string"}
SKAI_REPO = "https://github.com/google-research/skai.git"  #@param {type:"string"}
SKAI_BRANCH = "main"  #@param {type:"string"}
SKAI_COMMIT = "" #@param {type:"string"}

root_filesys=os.path.dirname(SKAI_CODE_DIR)

pathsys_venv=SKAI_VENV_DIR
pathsys_actenv=os.path.join(pathsys_venv, 'bin/activate')

pathsys_skai=SKAI_CODE_DIR
%shell rm -rf {SKAI_CODE_DIR}
%shell git clone -b {SKAI_BRANCH} {SKAI_REPO} {SKAI_CODE_DIR}
if SKAI_COMMIT!='':
  %shell cd {SKAI_CODE_DIR} ; git checkout {SKAI_COMMIT}

%cd {SKAI_CODE_DIR}/src/colab
import colab_utils
from colab_utils import *

def load_start_tensorboard(path_log):
  %load_ext tensorboard
  %tensorboard --logdir gs://{path_log}

colab_utils.load_start_tensorboard=load_start_tensorboard


In [ ]:
#@title Input project parameters

#@markdown Specify the variables to set your damage assessment project and press play:
#############################################
### INITIAL SETTING - PROJECT DESCRIPTION ###
#############################################
#@markdown ---
#@markdown Please enter here the parameters for your **disaster assessment project desciption:**

#@markdown ---
Disaster = 'Cyclone' #@param ["Cyclone", "Earthquake", "Tsunami", "Flood", "Eruption", "Tornado", "Wind", "Wildfire", "Landslide", "Conflict"]
Year =  None #@param {type:"integer"}
Month =  None #@param {type:"integer"}
Name = '' #@param {type:"string"}
Country = '' #@param {type:"string"}
Organisation = '' #@param {type:"string"}
Run = '' #@param {type:"string"}

disaster_mapping = {
    "Cyclone": "CYC",
    "Earthquake": "EQ",
    "Tsunami": "TSU",
    "Flood": "FLD",
    "Eruption": "RUP",
    "Tornado": "TOR",
    "Wind": "WND",
    "Wildfire": "WFL",
    "Landslide": "LSD",
    "Conflict": "CNF"
}

def get_iso3_code(country_name):
    # Check if the country name is in the PyCountry database
    try:
        country = pycountry.countries.lookup(country_name)
    except LookupError:
        # If the country name is not found, return None
        return country_name[:3]

    # Return the ISO 3 code for the country
    return country.alpha_3

PROJECT_DIRECTORY=f"{Organisation}-{disaster_mapping[Disaster]}-{Name}-{get_iso3_code(Country)}-{Year}{Month:02d}"
if not Run.isspace() and Run!="":
  PROJECT_DIRECTORY=f'{PROJECT_DIRECTORY}_{Run}'
PROJECT_DIRECTORY=f'{PROJECT_DIRECTORY}'.lower()


####################################################
### CLOUD SETTING - PROJECT/BUCKET CONFIGURATION ###
####################################################
#@markdown ---
#@markdown Please enter the parameters of **google cloud platform account** to use:

#@markdown ---
GCP_PROJECT = "" #@param {type:"string"}
GCP_LOCATION = "" #@param {type:"string"}
GCP_SERVICE_ACCOUNT=""#@param {type:"string"}

service_account = GCP_SERVICE_ACCOUNT
pathsys_credentials = '/root/service-account-private-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = pathsys_credentials

%shell gcloud config set project {GCP_PROJECT}
%shell gcloud config set compute/region {GCP_LOCATION}
%shell gcloud auth activate-service-account {GCP_SERVICE_ACCOUNT} --key-file={pathsys_credentials}

GCP_LOCATION_LABELING=GCP_LOCATION
if "europe-" in GCP_LOCATION :
  GCP_LOCATION_LABELING= "europe-west4"
  if GCP_LOCATION!= "europe-west1" :
    GCP_LOCATION= "europe-west1"
    print(f"\nLocation region has been changed to {GCP_LOCATION} (Vertex AI features availability) ")
if "us-" in GCP_LOCATION :
  GCP_LOCATION_LABELING= "us-central1"
  if GCP_LOCATION!= "us-central1" :
    GCP_LOCATION= "us-central1"
    print(f"\nLocation region has been changed to {GCP_LOCATION} (Vertex AI features availability) ")

GCP_PROJECT_ID=get_project_id(GCP_PROJECT)

#@markdown ---
#@markdown Please enter the parameters of **google cloud bucket** to create your working folder into your GCP project, named as

#@markdown *\<GCP_PROJECT>*colab_bucket_*\<BCKT_VERSION>*_*\<Author>*

#@markdown A specific folder for your disaster assessment project will be created under this bucket.

Tool="Colab"

#@markdown ---
BCKT_VERSION = "" #@param {type:"string"}
Author = '' #@param {type:"string"}

GCP_BUCKET = f"{GCP_PROJECT}_{Tool}_Bucket"
if not BCKT_VERSION.isspace() and BCKT_VERSION!="":
  GCP_BUCKET=f'{GCP_BUCKET}_{BCKT_VERSION}'
if not Author.isspace() and Author!="":
  GCP_BUCKET=f'{GCP_BUCKET}_{Author}'

GCP_BUCKET = GCP_BUCKET.lower()

if not bucket_exists(GCP_PROJECT, GCP_BUCKET):
  create_bucket(GCP_PROJECT, GCP_LOCATION, GCP_BUCKET)

print(f"\nYour project bucket in Google Cloud: {GCP_BUCKET} \nhttps://console.cloud.google.com/storage/browser/{GCP_BUCKET}")
print(f"\nYour project folder: {PROJECT_DIRECTORY}")

pathgcp_outputdir=os.path.join(GCP_BUCKET,PROJECT_DIRECTORY)

# Prepare credentials for map visualization.
credentials = ee.ServiceAccountCredentials(service_account, pathsys_credentials)
#Register the service account : https://signup.earthengine.google.com/#!/service_accounts
ee.Initialize(credentials)

pathsys_args={
    'python_env':pathsys_actenv,
    'path_skai':pathsys_skai,
    'path_cred':pathsys_credentials
}

#########################################
### IMAGE SETTING - FILE & DIRECTORY ###
#########################################
#@markdown ---
#@markdown Please enter the path to the files of **pre and post disaster satellite images** and **area of interest** you previously uploaded to your GCP project:

#@markdown ---
#IMAGERY_INPUT = "mosaic_images" #@param ["single_image", "mosaic_images"]
FILE_IMAGE_BEFORE = 'gs://bucket_path/*_Pre.tif' #@param {type:"string"}
FILE_IMAGE_AFTER = 'gs://bucket_path/*_Post.tif' #@param {type:"string"}
#@markdown Provide prefix of image filenames (replacing * in previous input), separated by commas (e.g. *Area1,Area2,Area3*). If consider all files, leave blank.
IMAGE_PREFIX_BEFORE = '' #@param {type:"string"}
IMAGE_PREFIX_AFTER = '' #@param {type:"string"}
FILE_IMAGE_AOI = 'gs://bucket_path/*.geojson' #@param {type:"string"}
#@markdown If you chose to use labeled file, please enter path to file, key and mapping to use:
FILE_IMAGE_LABELED = '' #@param {type:"string"}
KEY_IMAGE_LABELED = "" #@param {type:"string"}
MAPPING_IMAGE_LABELED = '' #@param {type:"string"}

#@markdown ---
#@markdown Choose where to get **building footprints** from:
BUILDING_DETECTION_METHOD = "open_buildings" #@param ["open_buildings","open_street_map","file"]
#@markdown If you chose "file", please enter path to CSV file here:
BUILDINGS_CSV = '' #@param {type:"string"}

pathgcp_imagesource=os.path.dirname(FILE_IMAGE_BEFORE).replace('gs://','')
pathgcp_images=os.path.join(pathgcp_outputdir,'images')

def get_list_images(prefixe_name,file_name):
  if prefixe_name=='':
    file_path_split=file_name.split('/')
    name_pattern='/'.join(file_path_split[3:])
    name_folder='/'.join(file_path_split[3:][:-1])+'/'
    url=f'https://storage.googleapis.com/storage/v1/b/{file_path_split[2]}/o?prefix={name_folder}'
    data = make_gcp_http_request(url)
    prefixe_name=','.join([re.search(name_pattern.replace('*','(.*)'),d['name']).group(1) for d in data['items'] if d['name']!= name_folder and re.search(name_pattern.replace('*','(.*)'),d['name'])])
  return ','.join([file_name.replace('*',prefix.strip()) for prefix in prefixe_name.split(',')])

pathgcp_imagebefore=get_list_images(IMAGE_PREFIX_BEFORE,FILE_IMAGE_BEFORE)
pathgcp_imageafter=get_list_images(IMAGE_PREFIX_AFTER,FILE_IMAGE_AFTER)

pathgcp_aoi=FILE_IMAGE_AOI
pathgcp_filelabeled=FILE_IMAGE_LABELED
pathgcp_keylabeled=KEY_IMAGE_LABELED
pathgcp_mappinglabeled=MAPPING_IMAGE_LABELED

#########################################
### EXAMPLE SETTING - CLOUD DIRECTORY ###
#########################################
pathgcp_examples=os.path.join(pathgcp_outputdir,'examples')
pathgcp_pattern=os.path.join(pathgcp_examples,'unlabeled-large/*.tfrecord')
pathgcp_importfolder=os.path.join(pathgcp_examples,'labeling_images')
pathgcp_importfile=os.path.join(pathgcp_importfolder,'import_file.csv')

###########################################
### LABELING SETTING - EMAIL PARAMETERS ###
###########################################
#@markdown ---
#@markdown Provide **email addresses** for all individuals that will help with labeling images, separated by commas.
#@markdown Emails of the labelers need to be linked to a google account.
#@markdown E.g. *manager@gmail.com* and *annotator1@gmail.com,annotator2@gmail.com*

#@markdown ---
EMAIL_MANAGER = '' #@param {type:"string"}
EMAIL_ANNOTATORS = '' #@param {type:"string"}

if EMAIL_MANAGER.strip() in EMAIL_ANNOTATORS:
  EMAIL_ANNOTATORS.replace(EMAIL_MANAGER.strip(), '')
GCP_LABELER_EMAIL = [EMAIL_MANAGER.strip()] + [email.strip() for email in EMAIL_ANNOTATORS.split(',')]
GCP_LABELER_EMAIL = ','.join(GCP_LABELER_EMAIL)

################################################
### DATASET SETTING - FILE & CLOUD DIRECTORY ###
################################################
pathgcp_temp=os.path.join(pathgcp_outputdir,'temp')
pathgcp_unlabeled=os.path.join(pathgcp_examples,'unlabeled/*.tfrecord')

pathgcp_trainset=os.path.join(pathgcp_examples,'labeled_train_examples.tfrecord')
pathgcp_testset=os.path.join(pathgcp_examples,'labeled_test_examples.tfrecord')

#######################################
### MODEL SETTING - FILE & DIRECTORY ##
#######################################
pathsys_runjobs=os.path.join(root_filesys,'run_jobs')
if not os.path.exists(pathsys_runjobs):
  os.mkdir(pathsys_runjobs)

pathgcp_models=os.path.join(pathgcp_outputdir,'models')


## Data labeling 👷

Create examples of buildings images before and after the disaster and classify them as either undamaged, possibly damaged, damaged/destroyed, or bad example (e.g., cloud cover etc.)

In [ ]:
#@title Visualize before and after images

display(Javascript("google.colab.output.resizeIframeToContent()"))
create_folium_map_with_images(pathgcp_imagebefore, pathgcp_imageafter)

In [ ]:
#@title Generate Examples
#@markdown First, generate the building images. This task should take about 30-45 minutes.

#@markdown Select if you want to generate your example from a labeled file or generate unlabeled examples.
GENERATING_JOB = "labeled" #@param ["labeled","unlabeled"]

## COMMAND RUN
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
GCP_DATASET_NAME = f"{Author}_example_{timestamp}_{PROJECT_DIRECTORY}".replace("_","-").lower()

if GENERATING_JOB == "labeled":
  generate_examples_args = {
    'cloud_project': GCP_PROJECT,
    'cloud_region': GCP_LOCATION,
    'dataset_name' : GCP_DATASET_NAME,
    'before_image_patterns': pathgcp_imagebefore,
    'after_image_patterns': pathgcp_imageafter,
    'aoi_path': pathgcp_aoi,
    'output_dir': f'gs://{pathgcp_outputdir}',
    'buildings_method': 'none',
    'worker_service_account': service_account,
    'earth_engine_service_account' : service_account,
    'earth_engine_private_key' : pathsys_credentials,
    'use_dataflow': 'true',
    'labels_file': pathgcp_filelabeled,
    'label_property': pathgcp_keylabeled,
    'labels_to_classes': pathgcp_mappinglabeled
    }

else:
  generate_examples_args = {
    'cloud_project': GCP_PROJECT,
    'cloud_region': GCP_LOCATION,
    'dataset_name' : GCP_DATASET_NAME,
    'before_image_patterns': pathgcp_imagebefore,
    'after_image_patterns': pathgcp_imageafter,
    'aoi_path': pathgcp_aoi,
    'output_dir': f'gs://{pathgcp_outputdir}',
    'buildings_method': BUILDING_DETECTION_METHOD,
    'buildings_file': BUILDINGS_CSV,
    'worker_service_account': service_account,
    'earth_engine_service_account' : service_account,
    'earth_engine_private_key' : pathsys_credentials,
    'use_dataflow': 'true',
    }


run_example_generation(generate_examples_args,pathsys_args,pretty_output=True)
print(f"\nGenerated examples are saved in the folder :\n{generate_examples_args['output_dir']}/examples\n")

In [ ]:
#@title Create Labeling Task

#@markdown Second, create the labeling tasks for the labelers. This task should take about 15-30 minutes.

#@markdown At the end of this step you and each labelers will receive an email with the instruction on how to perform the labeling task.

#@markdown Enter the maximum number of images to label (by default, 1000) :
MAX_IMAGES = 1000 #@param {type:"integer"}
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
GCP_DATASET_NAME = f"{Author}_label_{timestamp}_{PROJECT_DIRECTORY}".lower()
if MAX_IMAGES==0 or MAX_IMAGES is None:
  MAX_IMAGES=1000

create_labeling_task_args = {
    'cloud_project':GCP_PROJECT,
    'cloud_location':GCP_LOCATION_LABELING,
    'dataset_name': GCP_DATASET_NAME,
    'examples_pattern': f'gs://{pathgcp_pattern}',
    'images_dir':  f'gs://{pathgcp_importfolder}',
    'cloud_labeler_emails': GCP_LABELER_EMAIL,
    'max_images':MAX_IMAGES
    }

GCP_DATASET_ID ,GCP_DATASET_NAME,GCP_LABELING_JOB,GCP_LABELING_INSTRUCTION = run_labeling_task_creation(create_labeling_task_args,pathsys_args)

In [ ]:
#@title Monitor Labeling Task

#@markdown As a manager of the task, you can track the labeling progress by running this script below and see how many labels were created or view the detailed monitoring page.

#@markdown For good quality we recommend having about 200-300 building labels from the damaged/destroyed and undamaged categories.

#@markdown Enter a **labeling job** selection option. If you don't chose "runtime_saved", please enter the specific id of the job you would like to monitor.
LABELING_JOB = "runtime_saved" #@param ["runtime_saved","id"]
JOB_ID = "" #@param {type:"string"}

if LABELING_JOB=="id":
  GCP_LABELING_JOB=int(JOB_ID)
elif LABELING_JOB=="runtime_saved":
  if 'GCP_LABELING_JOB' not in locals():
    raise Exception('''
    Sorry, no Data Labeling job id is saved in your local runtine.
    Please change selection option and specify id of your data labeling job.''')

labeling_job = LabelingJob(f'{GCP_LOCATION_LABELING}-aiplatform.googleapis.com',
                           GCP_PROJECT, GCP_LOCATION_LABELING, GCP_LABELING_JOB)
print(f'\nJob completion percentage: {labeling_job.get_completion_percentage()}% (Data Labeling job ID {GCP_LABELING_JOB})')

## Training and evaluation datasets 🧩

Assign the labeled images to training and evaluation datasets.

In [ ]:
#@title Create training and evaluation datasets

#@markdown Enter a **labeling dataset** selection option. If you don't chose "runtime_saved", please enter the specific id of the dataset you would like to create your datasets.
LABELING_DATASET = "runtime_saved" #@param ["runtime_saved","id"]
DATASET_ID = "" #@param {type:"string"}

if LABELING_DATASET=="id":
  GCP_DATASET_ID=DATASET_ID
elif LABELING_DATASET=="runtime_saved":
  if 'GCP_DATASET_ID' not in locals():
    raise Exception('''
    Sorry, no Labeling dataset id is saved in your local runtine.
    Please change selection option and specify id of your labeling dataset.''')

create_labeled_dataset_args = {
    'cloud_project':GCP_PROJECT,
    'cloud_location':GCP_LOCATION_LABELING,
    'cloud_dataset_ids': GCP_DATASET_ID,
    'string_to_numeric_labels':'bad_example=0,no_damage=0,minor_damage=0,major_damage=1,destroyed=1',
    "cloud_temp_dir": f'gs://{pathgcp_temp}',
    "examples_pattern": f'gs://{pathgcp_unlabeled}',
    "train_output_path": f'gs://{pathgcp_trainset}',
    "test_output_path": f'gs://{pathgcp_testset}'}

create_labeled_dataset(create_labeled_dataset_args,pathsys_args)

### Inspect training and evaluation datasets (optional)

In [ ]:
#@title Inspect the training dataset (optional)

## COMMAND RUN
COUNT_TRAIN_LABELED=visualize_labeled_examples(os.path.join("gs://",pathgcp_trainset),max_examples=100)


In [ ]:
#@title Inspect the evaluation dataset (optional)

COUNT_TEST_LABELED=visualize_labeled_examples(os.path.join("gs://",pathgcp_testset),max_examples=100)

## Model training, performance evaluation 🤖

Train the machine learning model and test it using the evaluation dataset (leveraging the examples you previously labeled).

The script runs in the background and may take up to 6 hours. You will be able to see the progress on this page and we will also send you an email when this step is done.

In [ ]:
#@title Train and evaluate model

display(Javascript("google.colab.output.resizeIframeToContent()"))

#@markdown Select if you want to train a model on supervised or semi-supervised learning approach.
LEARNING_METHOD = "semi_supervised" #@param ["fully_supervised","semi_supervised"]

#@markdown View Results in Tensorboard:
LOAD_TENSORBOARD = 'Yes' #@param ["Yes","No"]

timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
GCP_EXPERIMENT_NAME=f"{Author}_experiment_{timestamp}_{PROJECT_DIRECTORY}_default".lower()
GCP_TRAINJOB_NAME=f"{Author}_train_{timestamp}_{PROJECT_DIRECTORY}_default".lower()
GCP_EVALJOB_NAME=f"{Author}_eval_{timestamp}_{PROJECT_DIRECTORY}_default".lower()

pathgcp_exper=os.path.join(pathgcp_models, GCP_EXPERIMENT_NAME)

generate_script_train_args={
    'project':GCP_PROJECT,
    'location':GCP_LOCATION,
    'job_type':'train',
    'display_name':GCP_TRAINJOB_NAME,
    'dataset_name':GCP_EXPERIMENT_NAME,
    'train_worker_machine_type':'n1-highmem-16',
    'train_docker_image_uri_path':'gcr.io/disaster-assessment/ssl-train-uri',
    'service_account':service_account,
    'train_dir':f'gs://{pathgcp_exper}',
    'train_unlabel_examples':f'gs://{pathgcp_unlabeled}',
    'train_label_examples':f'gs://{pathgcp_trainset}',
    'test_examples':f'gs://{pathgcp_testset}'}

generate_script_eval_args={
    'project':GCP_PROJECT,
    'location':GCP_LOCATION,
    'job_type':'eval',
    'display_name':GCP_EVALJOB_NAME,
    'dataset_name':GCP_EXPERIMENT_NAME,
    'eval_docker_image_uri_path':'gcr.io/disaster-assessment/ssl-eval-uri',
    'service_account':service_account,
    'train_dir':f'gs://{pathgcp_exper}',
    'train_unlabel_examples':f'gs://{pathgcp_unlabeled}',
    'train_label_examples':f'gs://{pathgcp_trainset}',
    'test_examples':f'gs://{pathgcp_testset}'}

if LEARNING_METHOD == "fully_supervised":
  generate_script_train_args['method']='fully_supervised'
  generate_script_eval_args['method']='fully_supervised'

if LOAD_TENSORBOARD=='Yes':
  run_train_and_eval_job([generate_script_train_args,generate_script_eval_args],
                       pathsys_args,
                       EMAIL_MANAGER,
                       sleep=[60],
                       pretty_output=True,
                       load_tensorboard=True,
                       path_log_tensorboard=pathgcp_exper)
else:
  run_train_and_eval_job([generate_script_train_args,generate_script_eval_args],
                       pathsys_args,
                       EMAIL_MANAGER,
                       sleep=[60],
                       pretty_output=True)

## Inference prediction 🔮

Use the model and create the damage assessment. When it is done you will be shown the summary statistics for the disaster along with a map based visualization of the damaged buildings.

In [ ]:
#@title Run Inference

display(Javascript("google.colab.output.resizeIframeToContent()"))

#@markdown Enter a **experimentation job** selection option. If you don't chose "runtime_saved", please enter the specific name of the job you would like to use to run the inference.
EXPER_JOB = "runtine_saved" #@param ["runtine_saved","name"]
JOB_NAME = "" #@param {type:"string"}

#@markdown Enter a **evaluation job** selection option. If you don't chose "runtime_saved", please enter the specific name or id of the job you would like to use to run the inference.
EVAL_JOB = "runtine_saved" #@param ["runtine_saved","name","id"]
JOB_ID_NAME = "" #@param {type:"string"}

#@markdown Enter a **checkpoint** selection option. If you chose "index_number", please enter the specific index of the checkpoint you would like to use to run the inference.
MODEL_CHECKPOINT = "most_recent" #@param ["most_recent","top_auc_test","top_acc_test","index_number"]
INDEX_NUMBER = "" #@param {type:"string"}

if EXPER_JOB=="name":
  GCP_EXPERIMENT_NAME=JOB_NAME
elif EXPER_JOB=="runtine_saved":
  if 'GCP_EXPERIMENT_NAME' not in locals():
    raise Exception('''
    Sorry, no Experiment job name is saved in your local runtine.
    Please change selection option and specify name of your experiment job.''')
pathgcp_exper=os.path.join(pathgcp_models, GCP_EXPERIMENT_NAME)

if EVAL_JOB=="id":
  GCP_EVAL_JOB=int(JOB_ID_NAME)
elif EVAL_JOB=="name":
  GCP_EVALJOB_NAME=JOB_ID_NAME
  GCP_EVAL_JOB=get_train_eval_job_id(GCP_PROJECT,GCP_LOCATION, GCP_EVALJOB_NAME)
elif EVAL_JOB=="runtine_saved":
  if 'GCP_EVAL_JOB' not in locals():
    if 'GCP_EVALJOB_NAME' in locals():
      GCP_EVAL_JOB=get_train_eval_job_id(GCP_PROJECT,GCP_LOCATION, GCP_EVALJOB_NAME)
    else:
      raise Exception('''
    Sorry, no Evaluation job id or name is saved in your local runtine.
    Please change selection option and specify id or name of your evaluation job.''')

epoch = get_epoch_number(pathgcp_exper,GCP_EVAL_JOB,MODEL_CHECKPOINT, INDEX_NUMBER)

# Create inference script that will be run by child process.
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
GCP_INFERENCE_NAME=f"{Author}_inference_{timestamp}_{PROJECT_DIRECTORY}_default".lower()

generate_script_infer_args={
    'project':GCP_PROJECT,
    'location':GCP_LOCATION,
    'eval_docker_image_uri_path':'gcr.io/disaster-assessment/ssl-eval-uri',
    'service_account':service_account,
    'dataset_name':GCP_EXPERIMENT_NAME,
    'train_dir':'gs://'+pathgcp_exper,
    'test_examples':'gs://'+pathgcp_unlabeled,
    'display_name':GCP_INFERENCE_NAME,
    'eval_ckpt': 'gs://'+pathgcp_exper+'/checkpoints/model.ckpt-'+epoch,
    'eval_worker_machine_type':'n1-highmem-16',
    'save_predictions':True,
    'inference_mode':True,
    'job_type':'eval'
    }

run_inference_and_prediction_job(generate_script_infer_args,
                                 pathsys_args,
                                 epoch,
                                 pretty_output=True)

In [ ]:
#@title Visualize Inference

display(Javascript("google.colab.output.resizeIframeToContent()"))

create_folium_map('/tmp/predictions.geojson',pathgcp_imagebefore, pathgcp_imageafter)